# OrgProfiler: Cyto2 Segmentation & Feature Extraction Pipeline

This notebook demonstrates the complete workflow:
1. Load/select images (from zip or directory)
2. Segmentation using Cellpose cyto2
3. Feature extraction using existing analysis functions


In [4]:
# Imports
import os
import zipfile
import io
from pathlib import Path
from typing import List, Dict, Any, Optional
import math


import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from scipy import ndimage as ndi
from scipy.spatial import ConvexHull
from skimage import measure, morphology, segmentation
from skimage.morphology import disk
from skimage.measure import perimeter_crofton
from skimage.segmentation import clear_border

# Import functions from main.py
# Note: Adjust the import path based on your setup
import sys
sys.path.append('.')

try:
    from main import (
        build_segmentation_mask_cyto2,
        convert_rgb_to_grayscale_uint8,
        calculate_feret_features_from_points,
        estimate_background_from_ring,
    )
    print("✓ Successfully imported functions from main.py")
except ImportError as e:
    print(f"⚠ Warning: Could not import from main.py: {e}")
    print("Functions will be defined inline below.")


⚠ Warning: Could not import from main.py: No module named 'anyio'
Functions will be defined inline below.


## 1. Image Loading Functions


In [5]:
def load_images_from_zip(zip_path: str) -> List[Dict[str, Any]]:
    """
    Load images from a zip file.
    
    Args:
        zip_path: Path to the zip file
        
    Returns:
        List of dictionaries with 'filename' and 'image' (RGB numpy array)
    """
    images = []
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get list of image files
        image_extensions = {'.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp'}
        image_files = [f for f in zip_ref.namelist() 
                      if Path(f).suffix.lower() in image_extensions]
        
        print(f"Found {len(image_files)} images in zip file")
        
        for filename in image_files:
            try:
                # Read image from zip
                image_data = zip_ref.read(filename)
                # Open with PIL and convert to RGB
                img = Image.open(io.BytesIO(image_data)).convert('RGB')
                # Convert to numpy array
                img_array = np.array(img)
                
                images.append({
                    'filename': Path(filename).name,
                    'image': img_array,
                    'shape': img_array.shape
                })
                print(f"  ✓ Loaded: {Path(filename).name} ({img_array.shape[0]}x{img_array.shape[1]})")
            except Exception as e:
                print(f"  ✗ Failed to load {filename}: {e}")
    
    return images


def load_images_from_directory(directory_path: str) -> List[Dict[str, Any]]:
    """
    Load images from a directory.
    
    Args:
        directory_path: Path to directory containing images
        
    Returns:
        List of dictionaries with 'filename' and 'image' (RGB numpy array)
    """
    images = []
    image_extensions = {'.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp'}
    
    directory = Path(directory_path)
    image_files = [f for f in directory.iterdir() 
                   if f.suffix.lower() in image_extensions and f.is_file()]
    
    print(f"Found {len(image_files)} images in directory")
    
    for img_path in image_files:
        try:
            img = Image.open(img_path).convert('RGB')
            img_array = np.array(img)
            
            images.append({
                'filename': img_path.name,
                'image': img_array,
                'shape': img_array.shape
            })
            print(f"  ✓ Loaded: {img_path.name} ({img_array.shape[0]}x{img_array.shape[1]})")
        except Exception as e:
            print(f"  ✗ Failed to load {img_path.name}: {e}")
    
    return images


## 2. Load Images

Choose one of the options below to load your images:


In [6]:
# Option 1: Load from zip file
# images = load_images_from_zip('path/to/your/images.zip')

# Option 2: Load from directory
# images = load_images_from_directory('path/to/your/images')

# Option 3: Load a single image for testing
# img_path = 'path/to/single/image.png'
# img = np.array(Image.open(img_path).convert('RGB'))
# images = [{'filename': Path(img_path).name, 'image': img, 'shape': img.shape}]

# Example placeholder (replace with actual path)
print("Please uncomment and modify one of the options above to load your images")


Please uncomment and modify one of the options above to load your images


## 3. Segmentation using Cyto2


In [7]:
def segment_image_cyto2(
    image_rgb: np.ndarray,
    diameter: Optional[float] = None,
    flow_threshold: float = 0.4,
    cellprob_threshold: float = 0.0,
    min_size: int = 15,
    clear_border: bool = True
) -> np.ndarray:
    """
    Segment image using Cellpose cyto2 model.
    
    Returns:
        Binary mask where True indicates foreground (cells/objects)
    """
    try:
        # Use function from main.py if available
        mask = build_segmentation_mask_cyto2(
            image_rgb,
            diameter=diameter,
            flow_threshold=flow_threshold,
            cellprob_threshold=cellprob_threshold,
            min_size=min_size,
            clear_border_artifacts=clear_border
        )
        return mask
    except (ImportError, AttributeError):
        # Fallback: define inline if import failed
        from cellpose import models, core
        
        # Convert to grayscale
        grayscale = convert_rgb_to_grayscale_uint8(image_rgb)
        image_for_cellpose = grayscale.astype(np.uint8, copy=False)
        
        # Load model
        model = models.Cellpose(model_type="cyto2", gpu=core.use_gpu())
        
        # Run segmentation
        result = model.eval(
            image_for_cellpose,
            diameter=diameter,
            flow_threshold=flow_threshold,
            cellprob_threshold=cellprob_threshold,
            min_size=min_size,
            channels=[0, 0]
        )
        
        if isinstance(result, tuple):
            masks = result[0]
        else:
            masks = result
        
        # Convert to binary mask
        binary_mask = (masks > 0).astype(bool)
        
        if clear_border:
            binary_mask = clear_border(binary_mask)
        
        return binary_mask


In [8]:
# Segment all images
# NOTE: Uncomment and run after loading images in the previous cell
# segmented_images = []
# 
# for img_data in images:
#     print(f"Segmenting {img_data['filename']}...")
#     
#     mask = segment_image_cyto2(
#         img_data['image'],
#         diameter=None,  # Let Cellpose estimate
#         flow_threshold=0.4,
#         cellprob_threshold=0.0,
#         min_size=15,
#         clear_border=True
#     )
#     
#     segmented_images.append({
#         'filename': img_data['filename'],
#         'image': img_data['image'],
#         'mask': mask,
#         'num_objects': int(mask.sum() > 0)  # Simple check
#     })
#     
#     print(f"  ✓ Done. Foreground pixels: {mask.sum()}")
# 
# print(f"\n✓ Segmented {len(segmented_images)} images")


## 4. Visualize Segmentation Results


In [ ]:
def visualize_segmentation(image: np.ndarray, mask: np.ndarray, filename: str = ""):
    """Visualize original image with segmentation overlay."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(image)
    axes[0].set_title(f'Original: {filename}')
    axes[0].axis('off')
    
    # Mask
    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title('Segmentation Mask')
    axes[1].axis('off')
    
    # Overlay
    overlay = image.copy()
    # Draw boundaries
    boundaries = segmentation.find_boundaries(mask, mode='outer')
    overlay[boundaries] = [255, 0, 255]  # Magenta boundaries
    
    axes[2].imshow(overlay)
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()


# Visualize first image (or all if you prefer)
# NOTE: Uncomment after segmentation
# if segmented_images:
#     for seg_data in segmented_images[:3]:  # Show first 3
#         visualize_segmentation(
#             seg_data['image'],
#             seg_data['mask'],
#             seg_data['filename']
#         )


## 5. Feature Extraction

Extract features from segmented images using the same pipeline as main.py


In [ ]:
# Import helper functions (if not already imported)
try:
    from main import (
        calculate_feret_features_from_points,
        estimate_background_from_ring,
        convert_rgb_to_grayscale_uint8
    )
    print("✓ Using functions from main.py")
except ImportError:
    print("⚠ Defining helper functions inline...")
    
    def calculate_feret_features_from_points(points_xy: np.ndarray):
        """Calculate Feret diameter features from a set of points."""
        if points_xy.shape[0] < 2:
            return 0.0, 0.0, 0.0, 0.0, 0.0
        
        convex_hull = ConvexHull(points_xy)
        hull_points = points_xy[convex_hull.vertices]
        
        max_distance_squared = 0.0
        best_start_index = 0
        best_end_index = 0
        
        for i in range(len(hull_points)):
            for j in range(i + 1, len(hull_points)):
                delta_x = hull_points[j, 0] - hull_points[i, 0]
                delta_y = hull_points[j, 1] - hull_points[i, 1]
                distance_squared = delta_x * delta_x + delta_y * delta_y
                
                if distance_squared > max_distance_squared:
                    max_distance_squared = distance_squared
                    best_start_index = i
                    best_end_index = j
        
        max_feret = math.sqrt(max_distance_squared)
        feret_angle = math.degrees(math.atan2(
            hull_points[best_end_index, 1] - hull_points[best_start_index, 1],
            hull_points[best_end_index, 0] - hull_points[best_start_index, 0]
        ))
        feret_x = float(hull_points[best_start_index, 0])
        feret_y = float(hull_points[best_start_index, 1])
        
        def calculate_width_for_edge(point_0, point_1):
            edge_vector_x = point_1[0] - point_0[0]
            edge_vector_y = point_1[1] - point_0[1]
            edge_length = math.hypot(float(edge_vector_x), float(edge_vector_y))
            
            if edge_length == 0:
                return float("inf")
            
            normal_x = -edge_vector_y / edge_length
            normal_y = edge_vector_x / edge_length
            
            projections = hull_points @ np.array([normal_x, normal_y], dtype=float)
            return float(projections.max() - projections.min())
        
        min_feret = float("inf")
        for i in range(len(hull_points)):
            width = calculate_width_for_edge(hull_points[i], hull_points[(i + 1) % len(hull_points)])
            if width < min_feret:
                min_feret = width
        
        return float(max_feret), float(min_feret), float(feret_angle), feret_x, feret_y
    
    def estimate_background_from_ring(
        image_uint8: np.ndarray,
        foreground_mask: np.ndarray,
        ring_width_pixels: int = 20,
        method: str = "median"
    ) -> float:
        """Estimate background intensity from a ring around the foreground mask."""
        ring_radius = max(1, int(ring_width_pixels))
        background_region = ~foreground_mask
        
        if not background_region.any():
            return 0.0
        
        distance_from_foreground = ndi.distance_transform_edt(background_region)
        ring_mask = (distance_from_foreground > 0) & (distance_from_foreground <= ring_radius)
        
        background_values = image_uint8[ring_mask]
        if background_values.size == 0:
            background_values = image_uint8[~foreground_mask]
        if background_values.size == 0:
            return 0.0
        
        if method == "median":
            return float(np.median(background_values))
        else:
            return float(background_values.mean())
    
    def convert_rgb_to_grayscale_uint8(image_rgb: np.ndarray) -> np.ndarray:
        """Convert RGB image to grayscale."""
        red_channel = image_rgb[..., 0].astype(np.float32)
        green_channel = image_rgb[..., 1].astype(np.float32)
        blue_channel = image_rgb[..., 2].astype(np.float32)
        grayscale = 0.299 * red_channel + 0.587 * green_channel + 0.114 * blue_channel
        return np.clip(grayscale, 0, 255).astype(np.uint8)


In [ ]:
def extract_features_from_mask(
    image: np.ndarray,
    mask: np.ndarray,
    pixel_size_um: float = 1.0,
    invert_for_intensity: bool = False,
    background_mode: str = "ring",
    ring_px: int = 20
) -> Dict[str, Any]:
    """
    Extract features from a segmented image.
    
    Args:
        image: RGB image array
        mask: Binary segmentation mask
        pixel_size_um: Pixel size in micrometers
        invert_for_intensity: Whether to invert image for intensity calculation
        background_mode: "ring" or "inverse_of_composite"
        ring_px: Ring width in pixels for background estimation
        
    Returns:
        Dictionary of extracted features
    """
    H, W = mask.shape[:2]
    
    # Label mask and get region properties
    labels = measure.label(mask, connectivity=2)
    if labels.max() == 0:
        return {}
    
    # For now, use largest region or union of all regions
    # You can modify this based on your needs
    if labels.max() > 1:
        # Use largest region
        props_list = measure.regionprops(labels)
        largest = max(props_list, key=lambda r: r.area)
        mask_measured = (labels == largest.label)
        props = largest
    else:
        mask_measured = mask
        props = measure.regionprops(mask.astype(np.uint8))[0]
    
    # Basic geometric features
    area_px = float(mask_measured.sum())
    perim_px = float(perimeter_crofton(mask_measured, directions=4))
    
    major_px = float(props.major_axis_length or 0.0)
    minor_px = float(props.minor_axis_length or 0.0)
    angle = float(np.degrees(props.orientation or 0.0))
    solidity = float(getattr(props, "solidity", 0.0))
    
    minr, minc, maxr, maxc = props.bbox
    cy_px, cx_px = props.centroid
    
    # Feret features
    contours = measure.find_contours(mask_measured, 0.5)
    if contours:
        cont = max(contours, key=lambda c: c.shape[0])
        if cont.shape[0] > 4000:
            cont = cont[::4]
        pts = np.column_stack((cont[:, 1], cont[:, 0]))  # (x, y)
    else:
        pts = np.empty((0, 2))
    
    if pts.shape[0] >= 3:
        feret_px, minFeret_px, feretAngle, feretX_px, feretY_px = calculate_feret_features_from_points(pts)
    else:
        feret_px = minFeret_px = feretAngle = feretX_px = feretY_px = 0.0
    
    # Convert to µm
    px_um = float(pixel_size_um)
    area = area_px * (px_um ** 2)
    perim = perim_px * px_um
    cx, cy = cx_px * px_um, cy_px * px_um
    major, minor = major_px * px_um, minor_px * px_um
    feret, minFeret = feret_px * px_um, minFeret_px * px_um
    feretX, feretY = feretX_px * px_um, feretY_px * px_um
    
    bx_px, by_px = float(minc), float(minr)
    width_px, height_px = float(maxc - minc), float(maxr - minr)
    bx, by = bx_px * px_um, by_px * px_um
    width, height = width_px * px_um, height_px * px_um
    
    # Shape metrics
    circ = (4.0 * math.pi * area) / (perim * perim) if perim > 0 else 0.0
    ar = float(major / minor) if minor > 0 else 0.0
    roundness = float((4.0 * area) / (math.pi * major * major)) if major > 0 else 0.0
    equivalent_diameter = math.sqrt(4.0 * area / math.pi)
    
    # Intensity features
    grayscale_uint8 = convert_rgb_to_grayscale_uint8(image)
    intensity_image = (255 - grayscale_uint8).astype(np.uint8) if invert_for_intensity else grayscale_uint8
    intensity_values = intensity_image[mask_measured].astype(np.uint8)
    
    if intensity_values.size == 0:
        return {}
    
    mean_intensity = float(intensity_values.mean())
    median_intensity = float(np.median(intensity_values))
    min_intensity = float(intensity_values.min())
    max_intensity = float(intensity_values.max())
    std_dev_intensity = float(intensity_values.std(ddof=0))
    
    histogram = np.bincount(intensity_values, minlength=256)
    mode_intensity = float(np.argmax(histogram))
    
    if std_dev_intensity > 0:
        z_scores = (intensity_values.astype(np.float32) - mean_intensity) / std_dev_intensity
        skewness = float(np.mean(z_scores ** 3))
        kurtosis = float(np.mean(z_scores ** 4) - 3.0)
    else:
        skewness = kurtosis = 0.0
    
    raw_integrated_density = float(intensity_values.sum())
    integrated_density = raw_integrated_density
    
    center_of_mass_y_px, center_of_mass_x_px = ndi.center_of_mass(
        intensity_image,
        labels=mask_measured.astype(np.uint8),
        index=1
    )
    center_of_mass_x = float(center_of_mass_x_px * px_um)
    center_of_mass_y = float(center_of_mass_y_px * px_um)
    
    # Background estimation
    if background_mode == "ring":
        background_intensity = estimate_background_from_ring(
            intensity_image,
            mask_measured,
            ring_width_pixels=ring_px,
            method="median"
        )
    else:
        # Use inverse of composite
        roi_intensity = intensity_image[int(by_px):int(by_px+height_px), int(bx_px):int(bx_px+width_px)]
        roi_background_mask = ~mask_measured[int(by_px):int(by_px+height_px), int(bx_px):int(bx_px+width_px)]
        background_values = roi_intensity[roi_background_mask]
        background_intensity = float(np.median(background_values)) if background_values.size else 0.0
    
    # Corrected intensity metrics
    corrected_total_intensity = raw_integrated_density - background_intensity * area_px
    corrected_mean_intensity = mean_intensity - background_intensity
    corrected_min_intensity = min_intensity - background_intensity
    corrected_max_intensity = max_intensity - background_intensity
    
    centroid_to_center_of_mass_distance = math.hypot(
        float(center_of_mass_x - cx),
        float(center_of_mass_y - cy)
    )
    
    return {
        "area": area,
        "perim": perim,
        "mean": mean_intensity,
        "median": median_intensity,
        "mode": mode_intensity,
        "min": min_intensity,
        "max": max_intensity,
        "stdDev": std_dev_intensity,
        "skew": -skewness,
        "kurt": kurtosis,
        "x": cx,
        "y": cy,
        "xm": center_of_mass_x,
        "ym": center_of_mass_y,
        "bx": bx,
        "by": by,
        "width": width,
        "height": height,
        "major": major,
        "minor": minor,
        "angle": angle,
        "circ": circ,
        "ar": ar,
        "round": roundness,
        "solidity": solidity,
        "feret": feret,
        "minFeret": minFeret,
        "feretAngle": feretAngle,
        "feretX": feretX,
        "feretY": feretY,
        "eqDiam": equivalent_diameter,
        "rawIntDen": raw_integrated_density,
        "intDen": integrated_density,
        "corrTotalInt": corrected_total_intensity,
        "corrMeanInt": corrected_mean_intensity,
        "corrMinInt": corrected_min_intensity,
        "corrMaxInt": corrected_max_intensity,
        "centroidToCom": centroid_to_center_of_mass_distance,
        "bgRing": background_intensity,
    }


In [ ]:
# Extract features from all segmented images
# NOTE: Uncomment and run after segmentation
# all_features = []
# 
# for seg_data in segmented_images:
#     print(f"Extracting features from {seg_data['filename']}...")
#     
#     features = extract_features_from_mask(
#         seg_data['image'],
#         seg_data['mask'],
#         pixel_size_um=1.0,  # Adjust based on your microscopy setup
#         invert_for_intensity=False,  # Set to True for brightfield
#         background_mode="ring",
#         ring_px=20
#     )
#     
#     features['filename'] = seg_data['filename']
#     all_features.append(features)
#     
#     print(f"  ✓ Area: {features.get('area', 0):.2f} µm²")
#     print(f"  ✓ Circularity: {features.get('circ', 0):.3f}")
#     print(f"  ✓ Mean intensity: {features.get('mean', 0):.2f}")
# 
# print(f"\n✓ Extracted features from {len(all_features)} images")


## 6. Display Results Table


In [ ]:
# Display results as a table
# NOTE: Uncomment after feature extraction
# import pandas as pd
# 
# # Convert to DataFrame for easy viewing
# df_features = pd.DataFrame(all_features)
# 
# # Display summary
# print("\n" + "="*80)
# print("FEATURE EXTRACTION SUMMARY")
# print("="*80)
# print(df_features.describe())
# 
# # Display full table
# print("\n" + "="*80)
# print("ALL FEATURES")
# print("="*80)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)
# print(df_features.round(3))


## 7. Export Results


In [ ]:
# Export to CSV and JSON
# NOTE: Uncomment after feature extraction
# import json
# 
# # Export to CSV
# output_csv = 'cyto2_analysis_results.csv'
# df_features.to_csv(output_csv, index=False)
# print(f"✓ Results exported to {output_csv}")
# 
# # Optionally export to JSON
# output_json = 'cyto2_analysis_results.json'
# with open(output_json, 'w') as f:
#     json.dump(all_features, f, indent=2)
# print(f"✓ Results exported to {output_json}")


## Summary

This notebook demonstrates:
1. ✓ Loading images from zip files or directories
2. ✓ Segmentation using Cellpose cyto2 model
3. ✓ Feature extraction using the same functions as main.py
4. ✓ Visualization of results
5. ✓ Export of results to CSV/JSON

You can now integrate this pipeline into your server's `/analyze/cyto2` endpoint!
